<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/CNN/Face_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
@Author: Satwik Ram K

Face Emotion Detection using Tensorflow

"""
# Importing Libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [9]:
num_class = 6
batch_size = 16
img_size = (48, 48, 1)

In [ ]:
! unzip /content/fer2013_1.zip

In [54]:
train_data_dir = '/content/fer2013_1/train'
val_data_dir = '/content/fer2013_1/validation'

In [55]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Activation, Flatten

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    fill_mode = 'nearest',
    rescale = 1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
)

In [58]:
val_datagen = ImageDataGenerator(rescale=1./255)


In [59]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode = 'grayscale',
    target_size = img_size[:2],
    batch_size = batch_size,
    class_mode = 'categorical' 
)

Found 28273 images belonging to 6 classes.


In [60]:
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    color_mode = 'grayscale',
    target_size = img_size[:2],
    batch_size = batch_size,
    class_mode = 'categorical' 
)

Found 3534 images belonging to 6 classes.


In [61]:
from tensorflow.keras.layers import ELU


Building Model

In [72]:
def build_model():

  model = Sequential()

  model.add(Conv2D(32, (5, 5), padding = "same", input_shape = img_size))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #2: second CONV => RELU => CONV => RELU => POOL
  # layer set
  model.add(Conv2D(64, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(256, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #3: third CONV => RELU => CONV => RELU => POOL
  # layer set
  model.add(Conv2D(256, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(512, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #4: third CONV => RELU => CONV => RELU => POOL
  # layer set
  model.add(Conv2D(1024, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(1024, (5, 5), padding="same"))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))

  # Block #5: first set of FC => RELU layers
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  # Block #6: second set of FC => RELU layers
  model.add(Dense(2048))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  # Block #7: softmax classifier
  model.add(Dense(num_class))
  model.add(Activation("softmax"))

  return model

In [73]:
model = build_model()

In [74]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 48, 48, 32)        832       
_________________________________________________________________
activation_54 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_50 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 24, 24, 64)        51264     
_________________________________________________________________
activation_55 (Activation)   (None, 24, 24, 64)       

In [75]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("emotions.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)



In [76]:
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)


In [77]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]


In [78]:
# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])


In [79]:
nb_train_samples = 28291
nb_validation_samples = 3534
epochs = 10

In [ ]:
model.fit_generator(
    train_genrator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = val_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/10
1508/1768 [========================>.....] - ETA: 34:49 - loss: 2.4196 - accuracy: 0.1926